In [3]:
%run aws_dwh.py

An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.
An error occurred (ClusterAlreadyExists) when calling the CreateCluster operation: Cluster already exists
                 Key  \
0  ClusterIdentifier   
1  NodeType            
2  ClusterStatus       
3  MasterUsername      
4  DBName              
5  Endpoint            
6  VpcId               
7  NumberOfNodes       

                                                                                   Value  
0  dwhcluster                                                                             
1  dc2.large                                                                              
2  available                                                                              
3  iandwh                                                                                 
4  redshiftdwhdb                                                                          
5  {'Address'

In [1]:
%run create_tables.py 

drop
DROP TABLE IF EXISTS staging_events
drop
DROP TABLE IF EXISTS staging_songs
drop
DROP TABLE IF EXISTS fact_songplay
drop
DROP TABLE IF EXISTS dim_user
drop
DROP TABLE IF EXISTS dim_song
drop
DROP TABLE IF EXISTS dim_artist
drop
DROP TABLE IF EXISTS dim_time

CREATE TABLE IF NOT EXISTS staging_events
(
artist          VARCHAR,
auth            VARCHAR, 
firstName       VARCHAR,
gender          VARCHAR,   
itemInSession   INTEGER,
lastName        VARCHAR,
length          FLOAT,
level           VARCHAR, 
location        VARCHAR,
method          VARCHAR,
page            VARCHAR,
registration    BIGINT,
sessionId       INTEGER,
song            VARCHAR,
status          INTEGER,
ts              TIMESTAMP,
userAgent       VARCHAR,
userId          INTEGER
);


CREATE TABLE IF NOT EXISTS staging_songs
(
song_id            VARCHAR,
num_songs          INTEGER,
title              VARCHAR,
artist_name        VARCHAR,
artist_latitude    FLOAT,
year               INTEGER,
duration           FLOAT,

In [2]:
%run etl.py 

executing query 
COPY staging_events FROM 's3://udacity-dend/log_data'
    iam_role  'arn:aws:iam::186402456289:role/dwhRole'
    COMPUPDATE OFF region 'us-west-2'
    TIMEFORMAT as 'epochmillisecs'
    TRUNCATECOLUMNS BLANKSASNULL EMPTYASNULL
    FORMAT AS JSON 's3://udacity-dend/log_json_path.json';

executed query.
executing query 
COPY staging_songs FROM 's3://udacity-dend/song_data'
    iam_role  'arn:aws:iam::186402456289:role/dwhRole'
    COMPUPDATE OFF region 'us-west-2'
    FORMAT AS JSON 'auto' 
    TRUNCATECOLUMNS BLANKSASNULL EMPTYASNULL;

executed query.
executing query 
INSERT INTO fact_songplay(start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT DISTINCT to_timestamp(to_char(se.ts, '9999-99-99 99:99:99'),'YYYY-MM-DD HH24:MI:SS'),
                se.userId as user_id,
                se.level as level,
                ss.song_id as song_id,
                ss.artist_id as artist_id,
                se.sessionId as session_id,
         